In [2]:
# Required libraries to import
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import pickle
import per

## Training and storing GMM models for each phoneme

In [3]:
mfcc_dir = 'models/withenergy/mfcc_13_2/'
TEST_DF = 'features/t_13_mfcc.hdf'
ENERGY_FLAG = False # To include energy co-efficient or not

In [11]:
# Importing the hdf file got by generation from the import_timit.py file 
timit_train_df = pd.read_hdf(TRAIN_DF)

In [12]:
# Show the first 5 rows of the table
timit_train_df.head()

,features,labels
0,"[6.827545062911363, -31.824121059207688, -6.55...",sil
1,"[6.70766922870581, -25.256157140416256, -4.668...",sil
2,"[7.159189212079523, -23.604278562432256, -13.0...",sil
3,"[8.2792130032043, -22.048277126965345, -24.025...",sil
4,"[8.51029438049915, -23.544322286337103, -29.46...",sil


In [13]:
# Extracting features and labels from the train set
train_features = np.array(timit_train_df["features"].tolist())
train_labels = np.array(timit_train_df["labels"].tolist())

In [14]:
np.shape(train_features)

(1236543, 13)

In [17]:
# Array indices to delete
# TODO : create a flag variable

'''if not ENERGY_FLAG:

    rem = [0,12,24]

    for i in rem:
        train_features = np.delete(train_features,i,1)'''

'if not ENERGY_FLAG:\n\n    rem = [0,12,24]\n\n    for i in rem:\n        train_features = np.delete(train_features,i,1)'

In [15]:
train_features.shape

(1236543, 13)

In [16]:
# Shows a list of the unique labels in a sorted manner
sorted(list(set(train_labels)))

['',
 'aa',
 'ae',
 'ah',
 'aw',
 'ay',
 'b',
 'ch',
 'd',
 'dh',
 'dx',
 'eh',
 'er',
 'ey',
 'f',
 'g',
 'hh',
 'ih',
 'iy',
 'jh',
 'k',
 'l',
 'm',
 'n',
 'ng',
 'ow',
 'oy',
 'p',
 'r',
 's',
 'sh',
 'sil',
 't',
 'th',
 'uh',
 'uw',
 'v',
 'w',
 'y',
 'z']

In [ ]:
#Training a gmm for each phoneme of the label_list and storing it in a directory in a pickled format
for index,i in enumerate(sorted(list(set(train_labels)))):
 gmm = GaussianMixture(n_components=64,covariance_type='diag')
 print('Phoneme ',index+1,' : ',i)
 gmm.fit(train_features[train_labels == i])
 if(i==''):
     pickle.dump(gmm,open(mfcc_dir+'blank.pkl','wb'))
 else:
     pickle.dump(gmm,open(mfcc_dir+i+'.pkl','wb'))

In [ ]:
# Replacing the blank phoneme label with the word 'blank'
sortlabels = sorted(list(set(train_labels)))
sortlabels[0] = 'blank'
sortlabels

In [ ]:
# Creating array to store all models
gmm_models=[]

# Loading and storing all 39 models into array
for i in sortlabels:
    with open(mfcc_dir+i+'.pkl','rb') as pkl_file:
        gmm_models.append(pickle.load(pkl_file))

In [ ]:
len(gmm_models)

In [ ]:
# Training Accuracy
train_scores=[]
# MAP calculation for each of the 39 models. Finally an array of arrays is got
for i in range(len(gmm_models)):
    print("MAP calculation for GMM model of phoneme ",i+1,': ',sortlabels[i])
    train_scores.append(gmm_models[i].score_samples(train_features))

In [ ]:
# Predicting labels using argmax indices
pred_labels=np.array(sorted(list(set(train_labels))))[np.argmax((np.transpose(train_scores)),axis=1)]
# Accuracy
train_acc = (np.count_nonzero(pred_labels==train_labels)/len(train_labels))*100
print("Training Accuracy: ",train_acc,"%")